# Рекомендация тарифов

# План работы:

[1. Знакомство с данными](#step1)

[2. Деление данных на три выборки: обучающая, валидационная, тестовая](#step2)

[3. Исследование моделей и выбор наилучшей](#step3)
- [решающее дерево](#step3.1)
- [случайный лес](#step3.2)
- [логистическая регрессия](#step3.3)


[4. Тестирование модели на тестовой выборке](#step4)

# Шаг 1. Знакомство с данными <a id='step1'></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Пропусков нет, типы данных соотвтетсвуют содержанию.

Целевой признак, который нужно будет предсказать - **`is_ultra`**. Остальные признаки - **`calls`**, **`minutes`**, **`messages`**, **`mb_used`**. Количество объектов - 3124, нужно поделить на три выборки - обучающую, валидационную и тестовую.

# Шаг 2. Деление данных на три выборки  <a id='step2'></a>

Разобъем исходную выборку на три в соотношении 60% - обучающая, 20% валидационная, 20% - тестовая.

In [4]:
data_train, data_intermediate = train_test_split(data, test_size=0.40, random_state=12345)
data_valid, data_test = train_test_split(data_intermediate, test_size=0.50, random_state=12345)

In [5]:
data_train.shape

(1928, 5)

In [6]:
data_valid.shape

(643, 5)

In [7]:
data_test.shape

(643, 5)

Получили три выборки:
- data_train - обучающая - 1928 объектов, 
- data_valid - валидационная - 643 объекта, для проверки качества моделей и выбора наилучшей
- data_test - тестовая - 643 объекта, для окончательной проверки качества выбранной модели

# Шаг3. Исследование моделей <a id='step3'></a>

В нашей задаче будем использовать три алгоритма классификации: DecisionTreeClassifier,  RandomForestClassifier, LogisticRegression. 
Нам предстоит: обучение, подбор гиперпарметров, оценка качества моделей и выбор наилучшей.

In [8]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

 - **решающее дерево** <a id='step3.1'></a>

In [9]:
#обучение модели и проверка качества модели на валидационной выборке с разной глубиной дерева
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    print('max_depth =',depth,':',accuracy_score(target_valid,valid_predictions))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Наилучшая метрика качества среди моделей DecisionTreeClassifier со значением параметра max_depth = 3.

- **случайный лес** <a id='step3.2'></a>

In [10]:
#обучение модели и проверка качества модели на валидационной выборке с разным значением n_estimator
for n in range(10, 70, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=n)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    print('n_estimators =',n,':',accuracy_score(target_valid,valid_predictions))

n_estimators = 10 : 0.7853810264385692
n_estimators = 20 : 0.7869362363919129
n_estimators = 30 : 0.7838258164852255
n_estimators = 40 : 0.7838258164852255
n_estimators = 50 : 0.7916018662519441
n_estimators = 60 : 0.7853810264385692


Наилучшая метрика качества среди моделей RandomForestClassifier со значением параметра n_estimators = 50 

- **логистическая регрессия** <a id='step3.3'></a>

In [11]:
#обучение модели и проверка качества модели на валидационной выборке c разными параметрами solver
solver = ['newton-cg', 'lbfgs', 'liblinear']
for element in solver:
    model = LogisticRegression(random_state=12345, solver=element)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    print('solver =',element, ' accuracy_score = ', accuracy_score(target_valid,valid_predictions))

solver = newton-cg  accuracy_score =  0.7558320373250389
solver = lbfgs  accuracy_score =  0.7107309486780715
solver = liblinear  accuracy_score =  0.7589424572317263


/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


Наилучшая метрика качества среди моделей LogisticRegression со значением параметра solver = 'liblinear'

Итак имеем три модели со следующими показателями качества:

In [12]:
modelDecisionTree = DecisionTreeClassifier(random_state=12345, max_depth = 3)
modelDecisionTree.fit(features_train, target_train)
valid_predictions_DecisionTree = modelDecisionTree.predict(features_valid)
accuracy_score_DecisionTree = accuracy_score(target_valid,valid_predictions_DecisionTree)

modelRandomForest = RandomForestClassifier(random_state=12345, n_estimators=50)
modelRandomForest.fit(features_train, target_train)
valid_predictions_RandomForest = modelRandomForest.predict(features_valid)
accuracy_score_RandomForest = accuracy_score(target_valid,valid_predictions_RandomForest)

modelLogisticRegression = LogisticRegression(random_state=12345, solver=element)
modelLogisticRegression.fit(features_train, target_train)
valid_predictions_LogisticRegression = modelLogisticRegression.predict(features_valid)
accuracy_score_LogisticRegression = accuracy_score(target_valid,valid_predictions_LogisticRegression)

result = pd.DataFrame({'model': ['DecisionTree', 'RandomForest', 'LogisticRegression'], 'accuracy_score': [accuracy_score_DecisionTree, accuracy_score_RandomForest, accuracy_score_LogisticRegression]})
result

,model,accuracy_score
0,DecisionTree,0.785381
1,RandomForest,0.791602
2,LogisticRegression,0.758942


Самая лушая метрика качества у модели RandomForest.

# Шаг4. Проверка модели на тестовой выборке <a id='step4'></a>

Проверим обученные ранее модели на тестовой выборке:

In [13]:
test_predictions_DecisionTree = modelDecisionTree.predict(features_test)
accuracy_score_DecisionTree = accuracy_score(target_test,test_predictions_DecisionTree)

test_predictions_RandomForest = modelRandomForest.predict(features_test)
accuracy_score_RandomForest = accuracy_score(target_test,test_predictions_RandomForest)

test_predictions_LogisticRegression = modelLogisticRegression.predict(features_test)
accuracy_score_LogisticRegression = accuracy_score(target_test,test_predictions_LogisticRegression)

result = pd.DataFrame({'model': ['DecisionTree', 'RandomForest', 'LogisticRegression'], 'accuracy_score': [accuracy_score_DecisionTree, accuracy_score_RandomForest, accuracy_score_LogisticRegression]})
result

,model,accuracy_score
0,DecisionTree,0.779160
1,RandomForest,0.793157
2,LogisticRegression,0.740280


Итак, что мы имеем. На тестовой выборке результат подтвердился: самая лучшая модель среди исследуемых в данной задаче RandomForest с параметром n_estimators=50. Метрика качества на тестовой выборке такая же высокая как и на валидационной, чуть выше. Доля правильных ответов - 79.3%.